In [1]:
#dataset link: https://drive.google.com/drive/folders/1n67sVTTzye4jtLfk8n-sa2fH2gTx5Ywt?usp=share_link

In [1]:
import tensorflow as tf
import os
from keras_preprocessing.image import ImageDataGenerator
DATA_DIR = "../Dataset Korosi/"

In [2]:
## resep yang nemu di: https://www.kaggle.com/code/vortexkol/alexnet-cnn-architecture-on-tensorflow-beginner

# model_alexnet = tf.keras.models.Sequential([
#     tf.keras.layers.Conv2D(filters=128, kernel_size=(11,11), strides=(4,4), activation='relu', input_shape=(150,150,3)),
#     tf.keras.layers.BatchNormalization(),
#     tf.keras.layers.MaxPool2D(pool_size=(2,2)),

#     tf.keras.layers.Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), activation='relu', padding="same"),
#     tf.keras.layers.BatchNormalization(),
#     tf.keras.layers.MaxPool2D(pool_size=(3,3)),

#     tf.keras.layers.Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
#     tf.keras.layers.BatchNormalization(),

#     tf.keras.layers.Conv2D(filters=256, kernel_size=(1,1), strides=(1,1), activation='relu', padding="same"),
#     tf.keras.layers.BatchNormalization(),

#     tf.keras.layers.Conv2D(filters=256, kernel_size=(1,1), strides=(1,1), activation='relu', padding="same"),
#     tf.keras.layers.BatchNormalization(),
#     tf.keras.layers.MaxPool2D(pool_size=(2,2)),

#     tf.keras.layers.Flatten(),

#     tf.keras.layers.Dense(1024,activation='relu'),
#     tf.keras.layers.Dropout(0.5),
#     tf.keras.layers.Dense(1024,activation='relu'),
#     tf.keras.layers.Dropout(0.5),
#     tf.keras.layers.Dense(3,activation='softmax')  
# ])

## resep yang nemu di: https://medium.com/swlh/alexnet-with-tensorflow-46f366559ce8
def create_model(size):

    model_alexnet = tf.keras.models.Sequential()
    model_alexnet.add(tf.keras.layers.experimental.preprocessing.Resizing(size, size, interpolation="bilinear", input_shape=(size,size,3)))
    model_alexnet.add(tf.keras.layers.Conv2D(96, 11, strides=4, padding='same'))
    model_alexnet.add(tf.keras.layers.Lambda(tf.nn.local_response_normalization))
    model_alexnet.add(tf.keras.layers.Activation('relu'))
    model_alexnet.add(tf.keras.layers.MaxPooling2D(3, strides=2))
    model_alexnet.add(tf.keras.layers.Conv2D(256, 5, strides=4, padding='same'))
    model_alexnet.add(tf.keras.layers.Lambda(tf.nn.local_response_normalization))
    model_alexnet.add(tf.keras.layers.Activation('relu'))
    model_alexnet.add(tf.keras.layers.MaxPooling2D(3, strides=2))
    model_alexnet.add(tf.keras.layers.Conv2D(384, 3, strides=4, padding='same'))
    model_alexnet.add(tf.keras.layers.Activation('relu'))
    model_alexnet.add(tf.keras.layers.Conv2D(384, 3, strides=4, padding='same'))
    model_alexnet.add(tf.keras.layers.Activation('relu'))
    model_alexnet.add(tf.keras.layers.Conv2D(256, 3, strides=4, padding='same'))
    model_alexnet.add(tf.keras.layers.Activation('relu'))
    model_alexnet.add(tf.keras.layers.Flatten())
    model_alexnet.add(tf.keras.layers.Dense(4096, activation='relu'))
    model_alexnet.add(tf.keras.layers.Dropout(0.5))
    model_alexnet.add(tf.keras.layers.Dense(4096, activation='relu'))
    model_alexnet.add(tf.keras.layers.Dropout(0.5))
    model_alexnet.add(tf.keras.layers.Dense(3, activation='softmax'))

    return model_alexnet


In [6]:
model_alexnet = create_model(128)
model_alexnet.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resizing_3 (Resizing)       (None, 128, 128, 3)       0         
                                                                 
 conv2d_9 (Conv2D)           (None, 32, 32, 96)        34944     
                                                                 
 lambda_6 (Lambda)           (None, 32, 32, 96)        0         
                                                                 
 activation_9 (Activation)   (None, 32, 32, 96)        0         
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 15, 15, 96)       0         
 2D)                                                             
                                                                 
 conv2d_10 (Conv2D)          (None, 4, 4, 256)         614656    
                                                      

In [7]:
import matplotlib.pyplot as plt

In [8]:
f = open("alexnet_pengujian_batch_size.csv", "w")
f.write("batch_size,loss,val_loss,acc,val_acc")
f.close()

f = open("alexnet_pengujian_batch_size_model_summary.txt", "w")
f.write("--------==== hasil model summary dari percobaan resize ====--------")
f.close()


In [9]:
BATCH_SIZES = [8,16,24,32]
size = 224

for batchsize in BATCH_SIZES:
    model_alexnet = create_model(size)
    
    f = open("alexnet_pengujian_batch_size_model_summary.txt", "a")
    f.write("\n Ukuran batch: "+str(batchsize)+" \n"+
       str(model_alexnet.summary())
    )
    f.close()

    Wsave = model_alexnet.get_weights()
    tf.keras.utils.plot_model(model_alexnet,to_file=str(batchsize)+"alexnet.png")    
    training_datagen = ImageDataGenerator(
        rescale = 1./255,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest',
        validation_split=0.2
        )

    val_datagen = ImageDataGenerator(
        rescale = 1./255,
        
    )

    train_generator = training_datagen.flow_from_directory(DATA_DIR, batch_size=batchsize,
                                                            class_mode='categorical',
                                                            target_size=(size,size))   
    val_generator = train_generator = training_datagen.flow_from_directory(DATA_DIR, 
                                                            subset='validation', 
                                                            batch_size=batchsize,
                                                            class_mode='categorical',
                                                            target_size=(size,size))
    model_alexnet.set_weights(Wsave)
    model_alexnet.compile(optimizer='adam', loss=tf.keras.losses.CategoricalCrossentropy(), metrics=['acc'])
    history = model_alexnet.fit(train_generator,validation_data=val_generator,epochs=50, verbose = 2)

    f = open("alexnet_pengujian_batch_size.csv", "a")
    f.write("\n"+str(batchsize)+","+
        str(history.history['loss'][-1])+","+
        str(history.history['val_loss'][-1])+","+
        str(history.history['acc'][-1])+","+
        str(history.history['val_acc'][-1])
    )
    f.close()
     
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Loss Model')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.savefig('./alexnet/loss_'+str(batchsize)+'.png')
    plt.cla()
    plt.plot(history.history['acc'])
    plt.plot(history.history['val_acc'])
    plt.title('Model Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.savefig('./alexnet/acc_'+str(batchsize)+'.png')
    
    plt.clf()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resizing_4 (Resizing)       (None, 128, 128, 3)       0         
                                                                 
 conv2d_14 (Conv2D)          (None, 32, 32, 96)        34944     
                                                                 
 lambda_8 (Lambda)           (None, 32, 32, 96)        0         
                                                                 
 activation_14 (Activation)  (None, 32, 32, 96)        0         
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 15, 15, 96)       0         
 2D)                                                             
                                                                 
 conv2d_15 (Conv2D)          (None, 4, 4, 256)         614656    
                                                      

2022-12-16 02:58:25.196903: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-12-16 02:58:25.606858: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-12-16 02:58:31.465782: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


4/4 - 10s - loss: 1.0947 - acc: 0.4167 - val_loss: 1.0587 - val_acc: 0.4833 - 10s/epoch - 2s/step
Epoch 2/50
4/4 - 6s - loss: 1.0692 - acc: 0.4333 - val_loss: 1.0448 - val_acc: 0.4833 - 6s/epoch - 2s/step
Epoch 3/50
4/4 - 6s - loss: 1.0355 - acc: 0.5000 - val_loss: 1.0382 - val_acc: 0.4833 - 6s/epoch - 2s/step
Epoch 4/50
4/4 - 7s - loss: 1.0481 - acc: 0.5000 - val_loss: 1.0384 - val_acc: 0.4833 - 7s/epoch - 2s/step
Epoch 5/50
4/4 - 7s - loss: 1.0282 - acc: 0.5167 - val_loss: 1.0379 - val_acc: 0.4833 - 7s/epoch - 2s/step
Epoch 6/50
4/4 - 7s - loss: 1.0479 - acc: 0.4500 - val_loss: 1.0386 - val_acc: 0.4833 - 7s/epoch - 2s/step
Epoch 7/50
4/4 - 7s - loss: 1.0823 - acc: 0.4667 - val_loss: 1.0393 - val_acc: 0.4833 - 7s/epoch - 2s/step
Epoch 8/50
4/4 - 7s - loss: 1.0630 - acc: 0.4833 - val_loss: 1.0412 - val_acc: 0.4833 - 7s/epoch - 2s/step
Epoch 9/50
4/4 - 7s - loss: 1.0519 - acc: 0.4833 - val_loss: 1.0413 - val_acc: 0.4833 - 7s/epoch - 2s/step
Epoch 10/50
4/4 - 7s - loss: 1.0666 - acc: 0.4

2022-12-16 03:04:03.824304: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-12-16 03:04:08.758919: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


4/4 - 9s - loss: 1.0946 - acc: 0.3167 - val_loss: 1.0617 - val_acc: 0.4833 - 9s/epoch - 2s/step
Epoch 2/50
4/4 - 7s - loss: 1.1020 - acc: 0.4333 - val_loss: 1.0460 - val_acc: 0.4833 - 7s/epoch - 2s/step
Epoch 3/50
4/4 - 7s - loss: 1.0472 - acc: 0.4833 - val_loss: 1.0434 - val_acc: 0.4833 - 7s/epoch - 2s/step
Epoch 4/50
4/4 - 7s - loss: 1.0314 - acc: 0.4833 - val_loss: 1.0393 - val_acc: 0.4833 - 7s/epoch - 2s/step
Epoch 5/50
4/4 - 7s - loss: 1.0402 - acc: 0.4833 - val_loss: 1.0386 - val_acc: 0.4833 - 7s/epoch - 2s/step
Epoch 6/50
4/4 - 7s - loss: 1.0176 - acc: 0.4833 - val_loss: 1.0383 - val_acc: 0.4833 - 7s/epoch - 2s/step
Epoch 7/50
4/4 - 7s - loss: 1.0802 - acc: 0.4833 - val_loss: 1.0383 - val_acc: 0.4833 - 7s/epoch - 2s/step
Epoch 8/50
4/4 - 7s - loss: 1.0543 - acc: 0.4833 - val_loss: 1.0397 - val_acc: 0.4833 - 7s/epoch - 2s/step
Epoch 9/50
4/4 - 7s - loss: 1.0414 - acc: 0.4833 - val_loss: 1.0394 - val_acc: 0.4833 - 7s/epoch - 2s/step
Epoch 10/50
4/4 - 7s - loss: 1.0610 - acc: 0.450

<Figure size 640x480 with 0 Axes>

In [15]:
def create_mobilenet(size):

    mobilenet = tf.keras.applications.mobilenet_v2.MobileNetV2(
    input_shape=(size,size,3),
    weights=None,
    include_top=False,
    )
    
    model_mobilenet = tf.keras.models.Sequential([
        tf.keras.layers.InputLayer( input_shape=(size,size,3)),
    mobilenet,
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(512,activation='relu'),
    tf.keras.layers.Dense(3,activation='softmax')
    ])
    return model_mobilenet

mobilenet = create_mobilenet(64) # best mobilenet
mobilenet.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_64 (Functi  (None, 2, 2, 1280)       2257984   
 onal)                                                           
                                                                 
 flatten_4 (Flatten)         (None, 5120)              0         
                                                                 
 dropout_8 (Dropout)         (None, 5120)              0         
                                                                 
 dense_12 (Dense)            (None, 512)               2621952   
                                                                 
 dense_13 (Dense)            (None, 3)                 1539      
                                                                 
Total params: 4,881,475
Trainable params: 4,847,363
Non-trainable params: 34,112
_______________________________________

In [16]:
f = open("mobilenet_pengujian_batch_size.csv", "w")
f.write("size,loss,val_loss,acc,val_acc")
f.close()

f = open("mobilenet_pengujian_batch_size_model_summary.txt", "w")
f.write("--------==== hasil model summary dari percobaan resize ====--------")
f.close()


In [17]:
BATCH_SIZES = [8,16,24,32]
size = 64

for batchsize in BATCH_SIZES:
    mobilenet = create_mobilenet(size)
    
    f = open("mobilenet_pengujian_batch_size_model_summary.txt", "a")
    f.write("\n Ukuran gambar: "+str(batchsize)+" \n"+
       str(mobilenet.summary())
    )
    f.close()

    Wsave = mobilenet.get_weights()
    tf.keras.utils.plot_model(mobilenet,to_file=str(batchsize)+"mobilenet.png")    
    training_datagen = ImageDataGenerator(
        rescale = 1./255,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest',
        validation_split=0.2
        )

    val_datagen = ImageDataGenerator(
        rescale = 1./255,
        
    )

    train_generator = training_datagen.flow_from_directory(DATA_DIR, batch_size=batchsize,
                                                            class_mode='categorical',
                                                            target_size=(size,size))   
    val_generator = train_generator = training_datagen.flow_from_directory(DATA_DIR, 
                                                            subset='validation', 
                                                            batch_size=batchsize,
                                                            class_mode='categorical',
                                                            target_size=(size,size))
    mobilenet.set_weights(Wsave)
    mobilenet.compile(optimizer='adam', loss=tf.keras.losses.CategoricalCrossentropy(), metrics=['acc'])
    history = mobilenet.fit(train_generator,validation_data=val_generator,epochs=50, verbose = 2)

    f = open("mobilenet_pengujian_batch_size.csv", "a")
    f.write("\n"+str(batchsize)+","+
        str(history.history['loss'][-1])+","+
        str(history.history['val_loss'][-1])+","+
        str(history.history['acc'][-1])+","+
        str(history.history['val_acc'][-1])
    )
    f.close()
     
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Loss Model')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.savefig('./mobilenet/loss_'+str(size)+'.png')
    plt.cla()
    plt.plot(history.history['acc'])
    plt.plot(history.history['val_acc'])
    plt.title('Model Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.savefig('./mobilenet/acc_'+str(size)+'.png')
    
    plt.clf()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_64 (Functi  (None, 2, 2, 1280)       2257984   
 onal)                                                           
                                                                 
 flatten_5 (Flatten)         (None, 5120)              0         
                                                                 
 dropout_9 (Dropout)         (None, 5120)              0         
                                                                 
 dense_14 (Dense)            (None, 512)               2621952   
                                                                 
 dense_15 (Dense)            (None, 3)                 1539      
                                                                 
Total params: 4,881,475
Trainable params: 4,847,363
Non-trainable params: 34,112
_______________________________________

2022-12-16 03:17:15.244109: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-12-16 03:17:29.108947: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


4/4 - 19s - loss: 7.5650 - acc: 0.2333 - val_loss: 1.0991 - val_acc: 0.2000 - 19s/epoch - 5s/step
Epoch 2/50
4/4 - 6s - loss: 0.8776 - acc: 0.7167 - val_loss: 1.0975 - val_acc: 0.4833 - 6s/epoch - 2s/step
Epoch 3/50
4/4 - 6s - loss: 1.2125 - acc: 0.6667 - val_loss: 1.0976 - val_acc: 0.4833 - 6s/epoch - 2s/step
Epoch 4/50
4/4 - 6s - loss: 0.9066 - acc: 0.7167 - val_loss: 1.0981 - val_acc: 0.4833 - 6s/epoch - 2s/step
Epoch 5/50
4/4 - 6s - loss: 0.5334 - acc: 0.7333 - val_loss: 1.0974 - val_acc: 0.4833 - 6s/epoch - 2s/step
Epoch 6/50
4/4 - 7s - loss: 0.9634 - acc: 0.7500 - val_loss: 1.0966 - val_acc: 0.4833 - 7s/epoch - 2s/step
Epoch 7/50
4/4 - 6s - loss: 0.3876 - acc: 0.8500 - val_loss: 1.0958 - val_acc: 0.4833 - 6s/epoch - 2s/step
Epoch 8/50
4/4 - 6s - loss: 0.5809 - acc: 0.7833 - val_loss: 1.0952 - val_acc: 0.4833 - 6s/epoch - 2s/step
Epoch 9/50
4/4 - 6s - loss: 0.5122 - acc: 0.7667 - val_loss: 1.0938 - val_acc: 0.4833 - 6s/epoch - 2s/step
Epoch 10/50
4/4 - 6s - loss: 0.5807 - acc: 0.7

2022-12-16 03:22:44.704404: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-12-16 03:22:52.184261: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


4/4 - 13s - loss: 12.8745 - acc: 0.4667 - val_loss: 1.1037 - val_acc: 0.3167 - 13s/epoch - 3s/step
Epoch 2/50
4/4 - 7s - loss: 2.9494 - acc: 0.7167 - val_loss: 1.0999 - val_acc: 0.2000 - 7s/epoch - 2s/step
Epoch 3/50
4/4 - 7s - loss: 4.4092 - acc: 0.6833 - val_loss: 1.0986 - val_acc: 0.2000 - 7s/epoch - 2s/step
Epoch 4/50
4/4 - 7s - loss: 4.6004 - acc: 0.7000 - val_loss: 1.0972 - val_acc: 0.4833 - 7s/epoch - 2s/step
Epoch 5/50
4/4 - 7s - loss: 1.9558 - acc: 0.5833 - val_loss: 1.0935 - val_acc: 0.4833 - 7s/epoch - 2s/step
Epoch 6/50
4/4 - 7s - loss: 1.6350 - acc: 0.7833 - val_loss: 1.0884 - val_acc: 0.4833 - 7s/epoch - 2s/step
Epoch 7/50
4/4 - 7s - loss: 1.1843 - acc: 0.8167 - val_loss: 1.0850 - val_acc: 0.4833 - 7s/epoch - 2s/step
Epoch 8/50
4/4 - 7s - loss: 1.1976 - acc: 0.8000 - val_loss: 1.0826 - val_acc: 0.4833 - 7s/epoch - 2s/step
Epoch 9/50
4/4 - 7s - loss: 0.9659 - acc: 0.7500 - val_loss: 1.0751 - val_acc: 0.4833 - 7s/epoch - 2s/step
Epoch 10/50
4/4 - 7s - loss: 0.7891 - acc: 0.

2022-12-16 03:28:22.919505: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-12-16 03:28:36.694661: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


4/4 - 19s - loss: 23.1535 - acc: 0.4667 - val_loss: 1.0903 - val_acc: 0.4833 - 19s/epoch - 5s/step
Epoch 2/50
4/4 - 8s - loss: 7.9034 - acc: 0.6833 - val_loss: 1.0855 - val_acc: 0.4833 - 8s/epoch - 2s/step
Epoch 3/50
4/4 - 8s - loss: 14.0061 - acc: 0.6333 - val_loss: 1.0630 - val_acc: 0.4833 - 8s/epoch - 2s/step
Epoch 4/50
4/4 - 8s - loss: 6.9069 - acc: 0.7333 - val_loss: 1.0908 - val_acc: 0.4833 - 8s/epoch - 2s/step
Epoch 5/50
4/4 - 7s - loss: 3.6337 - acc: 0.8000 - val_loss: 1.1812 - val_acc: 0.4833 - 7s/epoch - 2s/step
Epoch 6/50
4/4 - 7s - loss: 5.4505 - acc: 0.7500 - val_loss: 1.3995 - val_acc: 0.4833 - 7s/epoch - 2s/step
Epoch 7/50
4/4 - 7s - loss: 6.2556 - acc: 0.7500 - val_loss: 1.6131 - val_acc: 0.4833 - 7s/epoch - 2s/step
Epoch 8/50
4/4 - 7s - loss: 3.0368 - acc: 0.7833 - val_loss: 1.6747 - val_acc: 0.4833 - 7s/epoch - 2s/step
Epoch 9/50
4/4 - 7s - loss: 4.4470 - acc: 0.8333 - val_loss: 1.7266 - val_acc: 0.4833 - 7s/epoch - 2s/step
Epoch 10/50
4/4 - 7s - loss: 2.8399 - acc: 0

<Figure size 640x480 with 0 Axes>